In [12]:
import sys
sys.path.insert(0, '..')
print(sys.path)
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path+"/model")
print(sys.path)
import clip
import torch
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from PIL import Image
import pandas as pd
import numpy as np
import pytorch_lightning as pl
from pytorch_lightning import Trainer
from sklearn.metrics import top_k_accuracy_score
from sklearn.model_selection import train_test_split
import pickle
from tqdm import tqdm
import gc
import os
import pickle
from model.aliproduct_model import ALIPRODUCT_CLIP
from model.aliproduct_blip_model import ALIPRODUCT_BLIP
from model.dataset import ALIPRODUCT_DATASET,prepare_data
from model.CONFIG import CONFIG
import faiss
from torchvision import transforms
from torchvision.transforms.functional import InterpolationMode
import json

['..', '..', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference', '/home/ubuntu/anaconda3/lib/python39.zip', '/home/ubuntu/anaconda3/lib/python3.9', '/home/ubuntu/anaconda3/lib/python3.9/lib-dynload', '', '/home/ubuntu/anaconda3/lib/python3.9/site-packages', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/IPython/extensions', '/home/ubuntu/.ipython', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/model', '/tmp/tmph3xi4d1e']
['..', '..', '/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference', '/home/ubuntu/anaconda3/lib/python39.zip', '/home/ubuntu/anaconda3/lib/python3.9', '/home/ubuntu/anaconda3/lib/python3.9/lib-dynload', '', '/home/ubuntu/anaconda3/lib/python3.9/site-packages', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/locket-0.2.1-py3.9.egg', '/home/ubuntu/anaconda3/lib/python3.9/site-packages/IPython/extensions', '/home/ubuntu/.i

In [13]:
clean_df = pd.read_csv("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/code/test_set_inference/test.csv")
clean_df.head()

,Unnamed: 0,test_caption,test_imgs
0,0,feiyangg/LP Paragraph Style Electric Guitar Ti...,O1CN01r2RqZk1lJ4Sk2ZYaN_!!2200643934797.jpg
1,1,Summer Lace White Shorts Pants Female Thin Ant...,O1CN01H0o2pi2HAtGC9rVWa_!!2206736519111.jpg
2,2,High-Quality Export 8MM with Adhesive Cork Cor...,O1CN01qeHW0Z1D3mRPfmNvM_!!161-0-lubanu.jpg
3,3,LED Ceiling Living Room Bedroom Corridor Corri...,O1CN01phQoOe25ljjN1lpLO_!!3082327567.jpg
4,4,Disposable Cotton Cotton Female Thin Water Fac...,O1CN01Dzpgpr2FxJsUzqAdE_!!2963188946.jpg


In [14]:
image_embed_path = "./image_embeds"
text_embed_path = "./text_embeds"
image_embed_files = os.listdir(image_embed_path)
text_embed_files = os.listdir(text_embed_path)

In [15]:
all_image_embed ={}
all_text_embed = {}
for i,t in tqdm(zip(image_embed_files,text_embed_files)):
    with open(f"{image_embed_path}/{i}","rb") as img_file:
        img_embed = pickle.load(img_file)
        all_image_embed[img_embed["model_name"]] = img_embed["embed"]
    with open(f"{text_embed_path}/{t}","rb") as txt_file:
        txt_embed = pickle.load(txt_file)
        all_text_embed[txt_embed["model_name"]] = txt_embed["embed"]


8it [00:01,  6.88it/s]


In [16]:
all_image_embed.keys()

dict_keys(['base_1st_e_5', 'base_all_e_7', 'base_2nd_e_5', 'large_all_e_9'])

In [17]:
all_text_embed.keys()

dict_keys(['base_2nd_e_5', 'base_1st_e_5', 'base_all_e_7', 'large_all_e_9'])

In [18]:
pair_to_test = "large_all_e_9"

In [19]:
image_embed = all_image_embed[pair_to_test]
text_embed = all_text_embed[pair_to_test]

In [20]:
test  =text_embed @ image_embed.T

In [26]:
test_top_5 = torch.topk(test,5)
test_top_10 = torch.topk(test,10) 

In [22]:

def get_preds(all_image_embed,all_text_embed,pair_to_test):
    image_embed = all_image_embed[pair_to_test]
    text_embed = all_text_embed[pair_to_test]
    faiss_index = faiss.IndexFlatIP(256)
    faiss_index.add(image_embed.numpy().astype(np.float32))
    top5_k_e,top5_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),5)
    top10_k_e,top10_k_y_pred = faiss_index.search(text_embed.numpy().astype(np.float32),10)
    
    return top5_k_y_pred,top10_k_y_pred,top5_k_e,top10_k_e

In [23]:
top5_pred_idx,top10_pred_idx,top5_k_e,top10_k_e = get_preds(all_image_embed,all_text_embed,pair_to_test)

In [43]:
top10_pred_idx

array([[25753, 12449, 34138, ..., 34587, 10069, 22788],
       [38426, 39112, 32529, ..., 16172, 30800, 29620],
       [17054, 27432, 49020, ..., 24882, 12474, 31823],
       ...,
       [ 9803, 34500, 34051, ..., 31996, 43664, 23257],
       [47543,  8214, 45029, ..., 28000, 27068, 30109],
       [ 7565, 42316, 29315, ..., 33041,  1214, 12490]])

In [45]:
top10_k_e

array([[0.36663646, 0.366424  , 0.3638126 , ..., 0.34686798, 0.34638175,
        0.34442237],
       [0.40754303, 0.40176862, 0.35846546, ..., 0.33486727, 0.3344438 ,
        0.3322875 ],
       [0.31810096, 0.31236115, 0.31007776, ..., 0.2982954 , 0.29713687,
        0.29547036],
       ...,
       [0.3516692 , 0.3508915 , 0.3456747 , ..., 0.3298244 , 0.32647192,
        0.3231343 ],
       [0.37956727, 0.37740234, 0.3727898 , ..., 0.35726953, 0.3562709 ,
        0.35573325],
       [0.351683  , 0.32996088, 0.32360467, ..., 0.32028753, 0.31797275,
        0.31406122]], dtype=float32)

In [84]:
threshhold = [np.where(c > 0.3)[0] for c in top10_k_e]

In [85]:
top5_pred_idx_threshold = [top10_pred_idx[i][t][0:5] for i,t in enumerate(threshhold)]

In [82]:
top10_pred_idx_threshold = [top10_pred_idx[i][t] for i,t in enumerate(threshhold)]

In [28]:
def remap_pred_to_name(top5_pred_idx,top10_pred_idx,image_array):
    top5_pred = [list(image_array[p]) for p in top5_pred_idx]
    top10_pred =  [list(image_array[p]) for p in top10_pred_idx]
    return top5_pred,top10_pred

In [96]:
top5_pred,top10_pred =remap_pred_to_name(top5_pred_idx_threshold,top10_pred_idx_threshold,clean_df["test_imgs"].values)

In [38]:
clean_df["test_caption"].values[2]

'High-Quality Export 8MM with Adhesive Cork Cork Wallboard Background Wall Wall Kindergarten Cork Photo Wall'

In [99]:
# from PIL import Image
# _, axs = plt.subplots(2, 5, figsize=(12, 12))
# axs = axs.flatten()
# for img, ax in zip(top10_pred[2], axs):
#     image = Image.open("/home/ubuntu/Desktop/CVPR 2022 AliProducts Challenge/test_imgs/"+img)
#     ax.imshow(image)
# plt.show()

In [100]:
def results_to_json(top5_pred,top10_pred,caption_array,result_dir,result_file_name):
    results = []
    for c,t5,t10 in tqdm(zip(caption_array,top5_pred,top10_pred)):
        result_dict = {}
        result_dict["caption"] = c
        result_dict["top5"] = t5
        result_dict["top10"] = t10
        results.append(result_dict)
    print("checking results format")
    print(results[0])
    with open(f"{result_dir}/{result_file_name}.json","w") as results_file:
        json.dump(results,results_file)
    with open(f"{result_dir}/{result_file_name}.json","r") as results_file:
        check_results = json.load(results_file)
    print("checking results format of saved file")
    print(check_results[0])
    return check_results
    
        



In [102]:
clean_df

,Unnamed: 0,test_caption,test_imgs
0,0,feiyangg/LP Paragraph Style Electric Guitar Ti...,O1CN01r2RqZk1lJ4Sk2ZYaN_!!2200643934797.jpg
1,1,Summer Lace White Shorts Pants Female Thin Ant...,O1CN01H0o2pi2HAtGC9rVWa_!!2206736519111.jpg
2,2,High-Quality Export 8MM with Adhesive Cork Cor...,O1CN01qeHW0Z1D3mRPfmNvM_!!161-0-lubanu.jpg
3,3,LED Ceiling Living Room Bedroom Corridor Corri...,O1CN01phQoOe25ljjN1lpLO_!!3082327567.jpg
4,4,Disposable Cotton Cotton Female Thin Water Fac...,O1CN01Dzpgpr2FxJsUzqAdE_!!2963188946.jpg
...,...,...,...
49995,49995,this data is null,TB1gz7TgipE_u4jSZKbq6zCUVXa
49996,49996,this data is null,O1CN018DjGnX1QeV9jOECs2_!!3432982001.jpg
49997,49997,this data is null,O1CN019QiqKF1MnQTM0gdna_!!2211026631479.jpg
49998,49998,this data is null,TB1yui5VoT1gK0jSZFrq6ANCXXa


In [103]:
saved_result = results_to_json(top5_pred,top10_pred,clean_df["test_caption"].values[0:20000],"./submissions","threshold_testing_03")

20000it [00:00, 489288.58it/s]


checking results format
{'caption': 'feiyangg/LP Paragraph Style Electric Guitar Tiger Veneer Factory Direct Color Can Be Customized', 'top5': ['O1CN01aTuhTb1e6WOiqTLBe_!!755703822.jpg', 'O1CN01de6tah1n4zgxtDxiS_!!2212475335037-0-tblite.jpg', 'O1CN01oBYvbs1UCnjEaCYcZ_!!0-item_pic.jpg', 'TB154xYM.H1gK0jSZSyq6xtlpXa', 'O1CN01TpeBHn1aeB6BmIj6n_!!2212047703354-0-tblite.jpg'], 'top10': ['O1CN01aTuhTb1e6WOiqTLBe_!!755703822.jpg', 'O1CN01de6tah1n4zgxtDxiS_!!2212475335037-0-tblite.jpg', 'O1CN01oBYvbs1UCnjEaCYcZ_!!0-item_pic.jpg', 'TB154xYM.H1gK0jSZSyq6xtlpXa', 'O1CN01TpeBHn1aeB6BmIj6n_!!2212047703354-0-tblite.jpg', 'TB1sLgVejMZ7e4jSZFOq6x7epXa', 'O1CN01OQh7pZ1DFEL2yLy2Q_!!186-0-lubanu.jpg', 'O1CN01HkEkha26AT1lIcHQX_!!692927621.jpg', 'TB13XtSMfb2gK0jSZK9q6yEgFXa', 'O1CN01PiSx2P1vIkV1JnsPG_!!0-item_pic.jpg']}
checking results format of saved file
{'caption': 'feiyangg/LP Paragraph Style Electric Guitar Tiger Veneer Factory Direct Color Can Be Customized', 'top5': ['O1CN01aTuhTb1e6WOiqTLBe_!!7557